In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.collections as mcoll
import matplotlib.path as mpath

In [ ]:
%config InlineBackend.figure_format='retina'
plt.style.use("../../../mplstyles/paper_half.mplstyle")

In [ ]:
num_linking_snapshots = [2, 4, 6, 8, 12, 16, 24, 32]
timing_dmd = [2.977385233598761, 5.091184241400333, 6.7984917080029845, 7.410554508399218, 8.095092724799178, 8.899382191395853, 11.110102650208864, 11.947844975010957]

In [ ]:
# Implementation for the plotting comes from here: https://stackoverflow.com/questions/8500700/how-to-plot-a-gradient-color-line
def make_segments(x, y):
    """
    Create list of line segments from x and y coordinates, in the correct format
    for LineCollection: an array of the form numlines x (points per line) x 2 (x
    and y) array
    """

    points = np.array([x, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    return segments


def colorline(ax,
    x, y, z=None, cmap=plt.get_cmap('copper'), norm=plt.Normalize(0.0, 1.0),
        linewidth=3, alpha=1.0):
    """
    http://nbviewer.ipython.org/github/dpsanders/matplotlib-examples/blob/master/colorline.ipynb
    http://matplotlib.org/examples/pylab_examples/multicolored_line.html
    Plot a colored line with coordinates x and y
    Optionally specify colors in the array z
    Optionally specify a colormap, a norm function and a line width
    """

    # Default colors equally spaced on [0,1]:
    if z is None:
        z = np.linspace(0.0, 1.0, len(x))

    # Special case if a single number:
    if not hasattr(z, "__iter__"):  # to check for numerical input -- this is a hack
        z = np.array([z])

    z = np.asarray(z)

    segments = make_segments(x, y)
    lc = mcoll.LineCollection(segments, array=z, cmap=cmap, norm=norm,
                              linewidth=linewidth, alpha=alpha)

    ax.add_collection(lc)

    return lc


def plot_timing(ax, linkage, timing):
    path = mpath.Path(np.column_stack([linkage, timing]))
    verts = path.interpolated(steps=1).vertices
    x, y = verts[:, 0], verts[:, 1]
    z = np.linspace(0.3,1,len(num_linking_snapshots))
    colors = plt.cm.Blues(z)

    colorline(ax, x, y, z, cmap=plt.get_cmap('Blues'), linewidth=2)
    ax.scatter(x, y, c=colors, zorder=3)

    ax.set_xlabel(r"$N$")
    ax.set_ylabel("SpDMD application timing (s)")


In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1)
plot_timing(ax, num_linking_snapshots, timing_dmd)
ax.set_xlim([0, 33])
ax.set_ylim([0, 13])
plt.savefig("../figures/dmd_perf_counter.pdf",
                bbox_inches="tight", pad_inches=0.1,
                facecolor=None, edgecolor='auto'
                )